In [2]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
from tensorflow.keras.utils import to_categorical

In [3]:
text="""경마장에 있는 말이 뛰고 있다\n
그의 말이 법이다\n
가는 말이 고와야 오는 말이 곱다\n"""

In [4]:
t = Tokenizer()
t.fit_on_texts([text])
vocab_size = len(t.word_index) + 1
# 케라스 토크나이저의 정수 인코딩은 인덱스가 1부터 시작하지만,
# 케라스 원-핫 인코딩에서 배열의 인덱스가 0부터 시작하기 때문에
# 배열의 크기를 실제 단어 집합의 크기보다 +1로 생성해야하므로 미리 +1 선언 
print('단어 집합의 크기 : %d' % vocab_size)

단어 집합의 크기 : 12


In [5]:
print(t.word_index)

{'말이': 1, '경마장에': 2, '있는': 3, '뛰고': 4, '있다': 5, '그의': 6, '법이다': 7, '가는': 8, '고와야': 9, '오는': 10, '곱다': 11}


In [6]:
sequences = list()
for line in text.split('\n'): # Wn을 기준으로 문장 토큰화
    encoded = t.texts_to_sequences([line])[0]
    for i in range(1, len(encoded)):
        sequence = encoded[:i+1]
        sequences.append(sequence)

print('학습에 사용할 샘플의 개수: %d' % len(sequences))

학습에 사용할 샘플의 개수: 11


In [7]:
print(sequences)

[[2, 3], [2, 3, 1], [2, 3, 1, 4], [2, 3, 1, 4, 5], [6, 1], [6, 1, 7], [8, 1], [8, 1, 9], [8, 1, 9, 10], [8, 1, 9, 10, 1], [8, 1, 9, 10, 1, 11]]


In [8]:
max_len=max(len(l) for l in sequences) # 모든 샘플에서 길이가 가장 긴 샘플의 길이 출력
print('샘플의 최대 길이 : {}'.format(max_len))

샘플의 최대 길이 : 6


In [9]:
sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')

In [10]:
print(sequences)

[[ 0  0  0  0  2  3]
 [ 0  0  0  2  3  1]
 [ 0  0  2  3  1  4]
 [ 0  2  3  1  4  5]
 [ 0  0  0  0  6  1]
 [ 0  0  0  6  1  7]
 [ 0  0  0  0  8  1]
 [ 0  0  0  8  1  9]
 [ 0  0  8  1  9 10]
 [ 0  8  1  9 10  1]
 [ 8  1  9 10  1 11]]


In [11]:
sequences = np.array(sequences)
X = sequences[:,:-1]
y = sequences[:,-1]
# 리스트의 마지막 값을 제외하고 저장한 것은 X
# 리스트의 마지막 값만 저장한 것은 y. 이는 레이블에 해당됨.

In [12]:
print(X)

[[ 0  0  0  0  2]
 [ 0  0  0  2  3]
 [ 0  0  2  3  1]
 [ 0  2  3  1  4]
 [ 0  0  0  0  6]
 [ 0  0  0  6  1]
 [ 0  0  0  0  8]
 [ 0  0  0  8  1]
 [ 0  0  8  1  9]
 [ 0  8  1  9 10]
 [ 8  1  9 10  1]]


In [13]:
print(y) # 모든 샘플에 대한 레이블 출력

[ 3  1  4  5  1  7  1  9 10  1 11]


In [14]:
y = to_categorical(y, num_classes=vocab_size)

In [15]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, SimpleRNN

In [16]:
model = Sequential()
model.add(Embedding(vocab_size, 11, input_length=max_len-1)) # 레이블을 분리하였으므로 이제 X의 길이는 5
model.add(SimpleRNN(32))
model.add(Dense(vocab_size, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=200, verbose=2)

Epoch 1/200
1/1 - 1s - loss: 2.4887 - accuracy: 0.0000e+00
Epoch 2/200
1/1 - 0s - loss: 2.4713 - accuracy: 0.0000e+00
Epoch 3/200
1/1 - 0s - loss: 2.4541 - accuracy: 0.0909
Epoch 4/200
1/1 - 0s - loss: 2.4370 - accuracy: 0.2727
Epoch 5/200
1/1 - 0s - loss: 2.4199 - accuracy: 0.3636
Epoch 6/200
1/1 - 0s - loss: 2.4026 - accuracy: 0.3636
Epoch 7/200
1/1 - 0s - loss: 2.3851 - accuracy: 0.5455
Epoch 8/200
1/1 - 0s - loss: 2.3672 - accuracy: 0.5455
Epoch 9/200
1/1 - 0s - loss: 2.3488 - accuracy: 0.5455
Epoch 10/200
1/1 - 0s - loss: 2.3299 - accuracy: 0.4545
Epoch 11/200
1/1 - 0s - loss: 2.3103 - accuracy: 0.4545
Epoch 12/200
1/1 - 0s - loss: 2.2900 - accuracy: 0.3636
Epoch 13/200
1/1 - 0s - loss: 2.2688 - accuracy: 0.3636
Epoch 14/200
1/1 - 0s - loss: 2.2468 - accuracy: 0.3636
Epoch 15/200
1/1 - 0s - loss: 2.2239 - accuracy: 0.3636
Epoch 16/200
1/1 - 0s - loss: 2.2000 - accuracy: 0.3636
Epoch 17/200
1/1 - 0s - loss: 2.1753 - accuracy: 0.3636
Epoch 18/200
1/1 - 0s - loss: 2.1497 - accuracy: 

Epoch 147/200
1/1 - 0s - loss: 0.2364 - accuracy: 1.0000
Epoch 148/200
1/1 - 0s - loss: 0.2306 - accuracy: 1.0000
Epoch 149/200
1/1 - 0s - loss: 0.2249 - accuracy: 1.0000
Epoch 150/200
1/1 - 0s - loss: 0.2194 - accuracy: 1.0000
Epoch 151/200
1/1 - 0s - loss: 0.2140 - accuracy: 1.0000
Epoch 152/200
1/1 - 0s - loss: 0.2088 - accuracy: 1.0000
Epoch 153/200
1/1 - 0s - loss: 0.2037 - accuracy: 1.0000
Epoch 154/200
1/1 - 0s - loss: 0.1988 - accuracy: 1.0000
Epoch 155/200
1/1 - 0s - loss: 0.1941 - accuracy: 1.0000
Epoch 156/200
1/1 - 0s - loss: 0.1895 - accuracy: 1.0000
Epoch 157/200
1/1 - 0s - loss: 0.1850 - accuracy: 1.0000
Epoch 158/200
1/1 - 0s - loss: 0.1806 - accuracy: 1.0000
Epoch 159/200
1/1 - 0s - loss: 0.1764 - accuracy: 1.0000
Epoch 160/200
1/1 - 0s - loss: 0.1723 - accuracy: 1.0000
Epoch 161/200
1/1 - 0s - loss: 0.1683 - accuracy: 1.0000
Epoch 162/200
1/1 - 0s - loss: 0.1645 - accuracy: 1.0000
Epoch 163/200
1/1 - 0s - loss: 0.1607 - accuracy: 1.0000
Epoch 164/200
1/1 - 0s - loss: 

In [17]:
def sentence_generation(model, t, current_word, n): # 모델, 토크나이저, 현재 단어, 반복할 횟수
    init_word = current_word # 처음 들어온 단어도 마지막에 같이 출력하기위해 저장
    sentence = ''
    for _ in range(n): # n번 반복
        encoded = t.texts_to_sequences([current_word])[0] # 현재 단어에 대한 정수 인코딩
        encoded = pad_sequences([encoded], maxlen=5, padding='pre') # 데이터에 대한 패딩
        result = model.predict_classes(encoded, verbose=0)
    # 입력한 X(현재 단어)에 대해서 Y를 예측하고 Y(예측한 단어)를 result에 저장.
        for word, index in t.word_index.items(): 
            if index == result: # 만약 예측한 단어와 인덱스와 동일한 단어가 있다면
                break # 해당 단어가 예측 단어이므로 break
        current_word = current_word + ' '  + word # 현재 단어 + ' ' + 예측 단어를 현재 단어로 변경
        sentence = sentence + ' ' + word # 예측 단어를 문장에 저장
    # for문이므로 이 행동을 다시 반복
    sentence = init_word + sentence
    return sentence

In [18]:
print(sentence_generation(model, t, '경마장에', 4))
# '경마장에' 라는 단어 뒤에는 총 4개의 단어가 있으므로 4번 예측

c:\users\윤보경\appdata\local\programs\python\python38\lib\site-packages\tensorflow\python\keras\engine\sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


경마장에 있는 말이 뛰고 있다


In [19]:
print(sentence_generation(model, t, '그의', 2)) # 2번 예측

그의 말이 법이다


In [20]:
print(sentence_generation(model, t, '가는', 5)) # 5번 예측

가는 말이 고와야 오는 말이 곱다


In [21]:
import pandas as pd
from string import punctuation
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
from tensorflow.keras.utils import to_categorical

In [22]:
df=pd.read_csv('./ArticlesApril2018.csv')
df.head()

,articleID,articleWordCount,byline,documentType,headline,keywords,multimedia,newDesk,printPage,pubDate,sectionName,snippet,source,typeOfMaterial,webURL
0,5adf6684068401528a2aa69b,781,By JOHN BRANCH,article,Former N.F.L. Cheerleaders’ Settlement Offer: ...,"['Workplace Hazards and Violations', 'Football...",68,Sports,0,2018-04-24 17:16:49,Pro Football,"“I understand that they could meet with us, pa...",The New York Times,News,https://www.nytimes.com/2018/04/24/sports/foot...
1,5adf653f068401528a2aa697,656,By LISA FRIEDMAN,article,E.P.A. to Unveil a New Rule. Its Effect: Less ...,"['Environmental Protection Agency', 'Pruitt, S...",68,Climate,0,2018-04-24 17:11:21,Unknown,The agency plans to publish a new regulation T...,The New York Times,News,https://www.nytimes.com/2018/04/24/climate/epa...
2,5adf4626068401528a2aa628,2427,By PETE WELLS,article,"The New Noma, Explained","['Restaurants', 'Noma (Copenhagen, Restaurant)...",66,Dining,0,2018-04-24 14:58:44,Unknown,What’s it like to eat at the second incarnatio...,The New York Times,News,https://www.nytimes.com/2018/04/24/dining/noma...
3,5adf40d2068401528a2aa619,626,By JULIE HIRSCHFELD DAVIS and PETER BAKER,article,Unknown,"['Macron, Emmanuel (1977- )', 'Trump, Donald J...",68,Washington,0,2018-04-24 14:35:57,Europe,President Trump welcomed President Emmanuel Ma...,The New York Times,News,https://www.nytimes.com/2018/04/24/world/europ...
4,5adf3d64068401528a2aa60f,815,By IAN AUSTEN and DAN BILEFSKY,article,Unknown,"['Toronto, Ontario, Attack (April, 2018)', 'Mu...",68,Foreign,0,2018-04-24 14:21:21,Canada,"Alek Minassian, 25, a resident of Toronto’s Ri...",The New York Times,News,https://www.nytimes.com/2018/04/24/world/canad...


In [23]:
print('열의 개수: ',len(df.columns))
print(df.columns)

열의 개수:  15
Index(['articleID', 'articleWordCount', 'byline', 'documentType', 'headline',
       'keywords', 'multimedia', 'newDesk', 'printPage', 'pubDate',
       'sectionName', 'snippet', 'source', 'typeOfMaterial', 'webURL'],
      dtype='object')


In [24]:
df['headline'].isnull().values.any()

False

In [25]:
headline = [] # 리스트 선언
headline.extend(list(df.headline.values)) # 헤드라인의 값들을 리스트로 저장
headline[:5] # 상위 5개만 출력

['Former N.F.L. Cheerleaders’ Settlement Offer: $1 and a Meeting With Goodell',
 'E.P.A. to Unveil a New Rule. Its Effect: Less Science in Policymaking.',
 'The New Noma, Explained',
 'Unknown',
 'Unknown']

In [26]:
print('총 샘플의 개수 : {}'.format(len(headline))) # 현재 샘플의 개수

총 샘플의 개수 : 1324


In [27]:
headline = [n for n in headline if n != "Unknown"] # Unknown 값을 가진 샘플 제거
print('노이즈값 제거 후 샘플의 개수 : {}'.format(len(headline))) # 제거 후 샘플의 개수

노이즈값 제거 후 샘플의 개수 : 1214


In [28]:
headline[:5]

['Former N.F.L. Cheerleaders’ Settlement Offer: $1 and a Meeting With Goodell',
 'E.P.A. to Unveil a New Rule. Its Effect: Less Science in Policymaking.',
 'The New Noma, Explained',
 'How a Bag of Texas Dirt  Became a Times Tradition',
 'Is School a Place for Self-Expression?']

In [29]:
def repreprocessing(s):
    s=s.encode("utf8").decode("ascii",'ignore')
    return ''.join(c for c in s if c not in punctuation).lower() # 구두점 제거와 동시에 소문자화

text = [repreprocessing(x) for x in headline]
text[:5]

['former nfl cheerleaders settlement offer 1 and a meeting with goodell',
 'epa to unveil a new rule its effect less science in policymaking',
 'the new noma explained',
 'how a bag of texas dirt  became a times tradition',
 'is school a place for selfexpression']

In [30]:
t = Tokenizer()
t.fit_on_texts(text)
vocab_size = len(t.word_index) + 1
print('단어 집합의 크기 : %d' % vocab_size)

단어 집합의 크기 : 3494


In [31]:
sequences = list()

for line in text: # 1,214 개의 샘플에 대해서 샘플을 1개씩 가져온다.
    encoded = t.texts_to_sequences([line])[0] # 각 샘플에 대한 정수 인코딩
    for i in range(1, len(encoded)):
        sequence = encoded[:i+1]
        sequences.append(sequence)

sequences[:11] # 11개의 샘플 출력

[[99, 269],
 [99, 269, 371],
 [99, 269, 371, 1115],
 [99, 269, 371, 1115, 582],
 [99, 269, 371, 1115, 582, 52],
 [99, 269, 371, 1115, 582, 52, 7],
 [99, 269, 371, 1115, 582, 52, 7, 2],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372, 10],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372, 10, 1116],
 [100, 3]]

In [32]:
index_to_word={}
for key, value in t.word_index.items(): # 인덱스를 단어로 바꾸기 위해 index_to_word를 생성
    index_to_word[value] = key

print('빈도수 상위 582번 단어 : {}'.format(index_to_word[582]))

빈도수 상위 582번 단어 : offer


In [33]:
max_len=max(len(l) for l in sequences)
print('샘플의 최대 길이 : {}'.format(max_len))

샘플의 최대 길이 : 24


In [34]:
sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')
print(sequences[:3])

[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0   99  269]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0   99  269  371]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0   99  269  371 1115]]


In [35]:
sequences = np.array(sequences)
X = sequences[:,:-1]
y = sequences[:,-1]

In [36]:
y = to_categorical(y, num_classes=vocab_size)

In [37]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, LSTM

In [38]:
model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=max_len-1))
# y데이터를 분리하였으므로 이제 X데이터의 길이는 기존 데이터의 길이 - 1
model.add(LSTM(128))
model.add(Dense(vocab_size, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=200, verbose=2)

Epoch 1/200
244/244 - 3s - loss: 7.6347 - accuracy: 0.0286
Epoch 2/200
244/244 - 1s - loss: 7.1153 - accuracy: 0.0299
Epoch 3/200
244/244 - 1s - loss: 6.9774 - accuracy: 0.0340
Epoch 4/200
244/244 - 1s - loss: 6.8529 - accuracy: 0.0411
Epoch 5/200
244/244 - 1s - loss: 6.7132 - accuracy: 0.0433
Epoch 6/200
244/244 - 1s - loss: 6.5493 - accuracy: 0.0482
Epoch 7/200
244/244 - 1s - loss: 6.3871 - accuracy: 0.0536
Epoch 8/200
244/244 - 1s - loss: 6.1870 - accuracy: 0.0547
Epoch 9/200
244/244 - 1s - loss: 5.9919 - accuracy: 0.0613
Epoch 10/200
244/244 - 1s - loss: 5.8008 - accuracy: 0.0664
Epoch 11/200
244/244 - 1s - loss: 5.6208 - accuracy: 0.0698
Epoch 12/200
244/244 - 1s - loss: 5.4503 - accuracy: 0.0757
Epoch 13/200
244/244 - 1s - loss: 5.2895 - accuracy: 0.0811
Epoch 14/200
244/244 - 1s - loss: 5.1382 - accuracy: 0.0871
Epoch 15/200
244/244 - 1s - loss: 4.9923 - accuracy: 0.0946
Epoch 16/200
244/244 - 1s - loss: 4.8537 - accuracy: 0.1068
Epoch 17/200
244/244 - 1s - loss: 4.7193 - accura

244/244 - 1s - loss: 0.3227 - accuracy: 0.9150
Epoch 138/200
244/244 - 1s - loss: 0.3192 - accuracy: 0.9167
Epoch 139/200
244/244 - 1s - loss: 0.3165 - accuracy: 0.9168
Epoch 140/200
244/244 - 1s - loss: 0.3136 - accuracy: 0.9171
Epoch 141/200
244/244 - 1s - loss: 0.3116 - accuracy: 0.9180
Epoch 142/200
244/244 - 1s - loss: 0.3095 - accuracy: 0.9163
Epoch 143/200
244/244 - 1s - loss: 0.3101 - accuracy: 0.9168
Epoch 144/200
244/244 - 1s - loss: 0.3067 - accuracy: 0.9164
Epoch 145/200
244/244 - 1s - loss: 0.3026 - accuracy: 0.9191
Epoch 146/200
244/244 - 1s - loss: 0.3012 - accuracy: 0.9164
Epoch 147/200
244/244 - 1s - loss: 0.2967 - accuracy: 0.9162
Epoch 148/200
244/244 - 1s - loss: 0.2955 - accuracy: 0.9180
Epoch 149/200
244/244 - 1s - loss: 0.2978 - accuracy: 0.9163
Epoch 150/200
244/244 - 1s - loss: 0.2995 - accuracy: 0.9167
Epoch 151/200
244/244 - 1s - loss: 0.2952 - accuracy: 0.9152
Epoch 152/200
244/244 - 1s - loss: 0.2914 - accuracy: 0.9157
Epoch 153/200
244/244 - 1s - loss: 0.2

In [39]:
def sentence_generation(model, t, current_word, n): # 모델, 토크나이저, 현재 단어, 반복할 횟수
    init_word = current_word # 처음 들어온 단어도 마지막에 같이 출력하기위해 저장
    sentence = ''
    for _ in range(n): # n번 반복
        encoded = t.texts_to_sequences([current_word])[0] # 현재 단어에 대한 정수 인코딩
        encoded = pad_sequences([encoded], maxlen=23, padding='pre') # 데이터에 대한 패딩
        result = model.predict_classes(encoded, verbose=0)
    # 입력한 X(현재 단어)에 대해서 y를 예측하고 y(예측한 단어)를 result에 저장.
        for word, index in t.word_index.items(): 
            if index == result: # 만약 예측한 단어와 인덱스와 동일한 단어가 있다면
                break # 해당 단어가 예측 단어이므로 break
        current_word = current_word + ' '  + word # 현재 단어 + ' ' + 예측 단어를 현재 단어로 변경
        sentence = sentence + ' ' + word # 예측 단어를 문장에 저장
    # for문이므로 이 행동을 다시 반복
    sentence = init_word + sentence
    return sentence

In [40]:
print(sentence_generation(model, t, 'i', 10))
# 임의의 단어 'i'에 대해서 10개의 단어를 추가 생성

c:\users\윤보경\appdata\local\programs\python\python38\lib\site-packages\tensorflow\python\keras\engine\sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


i disapprove of school vouchers can i still apply for them
